In [1]:
import os
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-streaming-kafka-0-8_2.11:2.3.0 pyspark-shell'

import Geohash #pip install Geohash
import ast
import sys
import time
import json

from pymongo import MongoClient
from pyspark import SparkContext, SparkConf
from pyspark.streaming import StreamingContext
from pyspark.streaming.kafka import KafkaUtils

client = MongoClient()
db = client.fit5148_db
db.stream_model.drop()
db.climate_stream.drop()
db.hotspots_stream.drop()

In [2]:
def geohashlist(list):
    for i in list:
        lat = i['latitude']
        long = i['longitude']
        georesult = Geohash.encode(lat, long, precision = 5)
        i['geohash'] = georesult
    return list

def sendDataToDB(iter):
    client = MongoClient()
    db = client.fit5148_db
    clm = db.climate_stream
    hot = db.hotspots_stream
    model = db.stream_model
    list1 = []
    list2 = []
    list3 = []
    for record in iter:
        data = record[1].split(",",2)
        identifer = data[0]
        try:
            result = ast.literal_eval(data[2])
        except Exception as ex:
            print("Exception Occured. Message: {0}".format(str(ex)))
        if identifer == "1":
            list1.append(result)
            list1 = geohashlist(list1)
        if identifer == "2":
            list2.append(result)
            list2 = geohashlist(list2)
        if identifer == "3":
            list3.append(result)
            list3 = geohashlist(list3)
            
    count = 0
    if len(list2) == 1 and len(list3) == 1:
        if (list2[0]['geohash'] == list3[0]['geohash']):
            avgconfidence = float((list2[0]['confidence'] + list3[0]['confidence'])/2)
            avgtemp = float((list2[0]['surface_temperature_celcius'] + list3[0]['surface_temperature_celcius'])/2)
            list3[0]['confidence'] = avgconfidence
            list3[0]['surface_temperature_celcius'] = avgtemp
            count = 1
            hot.insert(list3[0])
            
    if len(list2) == 1:
        if count == 0:
            hot.insert(list2[0])
            
    if len(list3) == 1:
        if count == 0:
            hot.insert(list3[0])
                
    if len(list1) > 0:
        for i in list1:
            clm.insert(i)
            hot_spot_records = []
            if count == 1:
                if (i['geohash'] == list3[0]['geohash']):
                    hot_spot_records.append(list3[0])
                    i["hot_spot_records"] = hot_spot_records
            if count == 0:
                if len(list2) == 1:  
                    if (i['geohash'] == list2[0]['geohash']):
                        hot_spot_records.append(list2[0])
                        i["hot_spot_records"] = hot_spot_records
                if len(list3) == 1:
                    if (i['geohash'] == list3[0]['geohash']):
                        hot_spot_records.append(list3[0])
                        i["hot_spot_records"] = hot_spot_records
            model.insert(i)
            
    client.close()
    
n_secs = 10
topic = "TaskC"


conf = SparkConf().setAppName("KafkaStreamProcessor").setMaster("local[2]")
sc = SparkContext.getOrCreate()
if sc is None:
    sc = SparkContext(conf=conf)
sc.setLogLevel("WARN")
ssc = StreamingContext(sc, n_secs)
    
kafkaStream = KafkaUtils.createDirectStream(ssc, [topic], {
                        'bootstrap.servers':'127.0.0.1:9092', 
                        'group.id':'taskc', 
                        'fetch.message.max.bytes':'15728640',
                        'auto.offset.reset':'largest'})
                        # Group ID is completely arbitrary

lines = kafkaStream.foreachRDD(lambda rdd: rdd.foreachPartition(sendDataToDB))

ssc.start()
time.sleep(600) # Run stream for 10 minutes just in case no detection of producer
# ssc.awaitTermination()
ssc.stop(stopSparkContext=True,stopGraceFully=True)

KeyboardInterrupt: 